In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
df_tweet = pd.DataFrame()
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        filename = os.path.join(dirname,filename)
        if 'Tweets' in filename:
            df_tweet = df_tweet.append(pd.read_csv(filename))
            print('added : ',filename)
        

# Any results you write to the current directory are saved as output.

In [ ]:

df_hashtags = pd.read_csv('/kaggle/input/coronavirus-covid19-tweets/Hashtags.CSV')

In [ ]:
hash_tag_useful = df_hashtags.hashtag.value_counts().reset_index()


In [ ]:
 hash_tag_useful= hash_tag_useful[hash_tag_useful.hashtag>1000]

In [ ]:
hash_tag_useful.to_csv('hastags.csv')

In [ ]:
!pwd

In [ ]:
df_country = pd.read_csv('/kaggle/input/coronavirus-covid19-tweets/Countries.CSV')
df_country.head()

In [ ]:
df_country[df_country.country_code.isin(['US','GB','IN','ES','CA'])]

In [ ]:
print("Top 5 country tweets")
ctry = df_tweet.country_code.value_counts()
ctry.head(10)

In [ ]:
df_tweet.shape

In [ ]:
#Only on top 5
df_tweet=df_tweet[df_tweet.country_code.isin(['US','CA','IN','GB','ES'])].reset_index(drop=True)
df_tweet.shape

In [ ]:
df_tweet.columns

In [ ]:
df_tweet.verified.value_counts()

In [ ]:
df_tweet.place_full_name.value_counts()

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [ ]:
def sentiment_analyzer_Score(sentence):
    score = analyser.polarity_scores(sentence)
    return(score)

In [ ]:
df_tweet.head()

In [ ]:
# df_tweet['neg_score'] = 0
# df_tweet['pos_score'] = 0
# df_tweet['neu_score'] = 0
df_tweet['composite_score'] = 0
for i in range(0,len(df_tweet)-1):
    print(i)
    s = sentiment_analyzer_Score(df_tweet.iloc[i,4])
#     df_tweet.iloc[i,22] = s['neg']
#     df_tweet.iloc[i,23] = s['neu']
#     df_tweet.iloc[i,24] = s['pos']
    df_tweet.iloc[i,22] = s['compound']

In [ ]:
df_tweet.loc[df_tweet.composite_score>=0.05,'sentiment'] = 'positive'
df_tweet.loc[df_tweet.composite_score<-0.05,'sentiment'] = 'negative'
df_tweet.loc[(df_tweet.composite_score<0.05)&(df_tweet.composite_score>-0.05),'sentiment'] = 'neutral'
df_tweet.sentiment.value_counts()

In [ ]:
df_tweet.head()


In [ ]:
df_tweet['created_at'] = pd.to_datetime(df_tweet['created_at'])

In [ ]:
#df_tweet.loc[df_tweet.sentiment=='positive','created_at'].value_counts()
pd.crosstab(index=df_tweet['created_at'], columns=df_tweet['sentiment']).plot.bar(stacked=True,title='tweet sentiment top 5 countries (US|CA|IN|ES|GB)')

In [ ]:
pd.crosstab(index=df_india['created_at'], columns=df_india['sentiment']).plot.bar()

In [ ]:
df_india = df_tweet[df_tweet.country_code=='IN']
pd.crosstab(index=df_india['created_at'], columns=df_india['sentiment']).plot.bar(stacked=True,title='tweet sentiment for INDIA')

In [ ]:
pd.crosstab(index=df_tweet['country_code'], columns=df_tweet['sentiment']).plot.bar(stacked=True,title='tweet sentiment for top 5 countries (US|CA|IN|ES|GB)')

In [ ]:
df_tweet.columns


In [ ]:
df_tweet[df_tweet.country_code=='IN'].groupby(['place_type'])['sentiment'].count()

In [ ]:
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator

In [ ]:
def Text_Cleaning(data, remove_numbers = True, custom_stopwords = None , remove_stop_words=True, lemmatize_words = False, stem_words= False):

    """

    Text Cleaning  
    Input: List of Documents    Output: Cleaned List of Documents
    Parameters: 
    custom_stopwords - Custom List of Stopwords(should be passed as a list)
    remove_stop_words - Binary parameter (True/False) to remove stopwords or do not want to perform this. Default is True
    lemmatize_words - Binary parameter (True/False) for Lemmatization (WordNet Lemmatizer used) or not. Default is False
    stem_words - Binary parameter (True/False) for Stemming (Lancaster Stemmer used) or not. Default is False

    """
    import os

    package = "nltk"

    try:
        __import__package
    except:
        os.system("pip install "+ package)
        import nltk
        nltk.download('stopwords')
        nltk.download('wordnet')
    
    import nltk
    import pandas as pd
    import string
    
    

    import re
    data = [re.sub("^\s+","",x) for x in data] # removes leading white spaces
    data = [re.sub("\s+\Z","",x) for x in data] # removes trailing white spaces

    data = [x.lower() for x in data]
    data = [x.translate(str.maketrans(' ',' ', '!"#$%&\'()*+,./:;<=>?@[\\]^`{|}~_')) for x in data]
    data = [x.replace('-', ' ') for x in data]
    
    if remove_numbers:
        data = [x.translate(str.maketrans('','', string.digits)) for x in data]


    ### Tokenizer
    def tokenize_word_text(text):
        """
        Sentence tokenize in NLTK with sent_tokenize 
        The sent_tokenize function uses an instance of NLTK known as PunktSentenceTokenizer
        This instance of NLTK has already been trained to perform tokenization on different European languages on the basis 
        of letters or punctuation that mark the beginning and end of sentences
        """
        from nltk.tokenize import WordPunctTokenizer 
        tokenizer = WordPunctTokenizer()
        return tokenizer.tokenize(text)


    ## Removing Stopwords
    if remove_stop_words:
        roman_numerals = ['iii', 'iiv', 'iiiv', 'vii', 'viii', 'xii', 'xiii', 'xxi','xxii', 'xxiii', 'xxv', 'xxvi', 'xxvii'] #exclude 2char numerals
        temporal_words = ['coronavirus','india','covid_19','hour', 'day', 'week', 'month', 'quarter', 'year', 'annual', 'today', 'tomorrow', 'yesterday', 
                          'prior', 'prio', 'preceed', 'yet', "before", 'after', 'earlier', 'later', 'ago', 'this', 'next', 'last',
                          'once-daili', 'once-month', 'once-through', 'once-week', 'once', 'also','covid 19','corona','co',                          
                          'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday','coronaviru','http','http co',
                          'jan', 'feb', 'apr', 'jun', 'jul', 'aug', 'sept', 'oct', 'nov', 'dec', 'may', 'onmay', 'postmay'
                         ,'corona virus','pandemic']
        from nltk.corpus import stopwords
        stopword_list = stopwords.words('english')
        if custom_stopwords:
            stop_list = set(stopword_list + custom_stopwords + roman_numerals + temporal_words)
        else:
            stop_list = set(stopword_list + roman_numerals + temporal_words)

        def removeStopwords_before_tokens(text, stopwords):
            tokens = tokenize_word_text(text)
            return ' '.join([w for w in tokens if w not in stopwords])

        data = [removeStopwords_before_tokens(x, stop_list) for x in data]



    ##### Word Lemmatization Using WordNet
    def wordlemmatize(text):
        from nltk.stem.wordnet import WordNetLemmatizer 
        lem = WordNetLemmatizer()
        tokens = tokenize_word_text(text)
        return ' '.join([lem.lemmatize(w) for w in tokens])
    if lemmatize_words:
        data = [wordlemmatize(x) for x in data]



    ##### Word Stemming Using Lancaster Stemmer
    def wordstem(text):
        from nltk.stem.porter import PorterStemmer
        ls = PorterStemmer()
        tokens = tokenize_word_text(text)
        return ' '.join([ls.stem(w) for w in tokens])
    if stem_words:
        data = [wordstem(x) for x in data]


   
    return data

In [ ]:
df_tweet['clean_tweet'] = Text_Cleaning(df_tweet['text'],lemmatize_words = True)

In [ ]:
sub_economy = df_tweet[df_tweet.clean_tweet.str.contains('economy')]
sub_economy.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
# Start with one review:
df_temp = df_tweet
df_ADR = df_temp[df_temp['sentiment']=='positive']
df_NADR = df_temp[df_temp['sentiment']=='negative']
tweet_All = " ".join(review for review in df_temp.clean_tweet)
tweet_ADR = " ".join(review for review in df_ADR.clean_tweet)
tweet_NADR = " ".join(review for review in df_NADR.clean_tweet)

fig, ax = plt.subplots(3, 1, figsize  = (30,30))
# Create and generate a word cloud image:
wordcloud_ALL = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_All)
wordcloud_ADR = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_ADR)
wordcloud_NADR = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_NADR)

# Display the generated image:
ax[0].imshow(wordcloud_ALL, interpolation='bilinear')
ax[0].set_title('All Tweets', fontsize=30)
ax[0].axis('off')
ax[1].imshow(wordcloud_ADR, interpolation='bilinear')
ax[1].set_title('Tweets under positive Class',fontsize=30)
ax[1].axis('off')
ax[2].imshow(wordcloud_NADR, interpolation='bilinear')
ax[2].set_title('Tweets under negative Class',fontsize=30)
ax[2].axis('off')


In [ ]:
wordcloud.to_file("img/first_review_usa.png")